In [1]:
import torch
from torch.amp import autocast, GradScaler
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import os
from tqdm import tqdm
from experiments.Checkpointing import save_checkpoint
from models.VAE import VAE
from modules.Losses import VAEVggLoss
from modules.SaveOutputs import save_reconstructions

torch.backends.cudnn.benchmark = True
torch.cuda.empty_cache()

dataset_dir: str = "./data/raw"
out_dir: str = "./outputs/"
batch_size: int = 64
latent_dim: int = 256
checkpoint_dir = "./experiments/checkpoints"
epochs: int = 10
lr: float = 3e-4
beta: float = 0.5

torch.manual_seed(0)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

transform = transforms.Compose([
    transforms.Resize((64, 64)),  # resize to 224x224
    transforms.ToTensor()  # convert to tensor & scale to [0,1]
])

dataset = datasets.ImageFolder(root="G:\Temp", transform=transform)
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [int(0.95*len(dataset)), len(dataset) - int(0.95*len(dataset))])
train_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True, num_workers=2, pin_memory=True)
n_train = len(train_loader.dataset)

print("Loaded datasets, number of samples: ", n_train)

# Model & Optimizer
scaler = GradScaler(device)
model = VAE(latent_dim=latent_dim).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
vgg_loss = VAEVggLoss()

current_epoch = 0
# os.removedirs(out_dir)
os.makedirs(out_dir, exist_ok=True)

Loaded datasets, number of samples:  10130


D:\Software\Conda\envs\Torch\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
D:\Software\Conda\envs\Torch\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
epochs = 50
for epoch in range(current_epoch + 1, current_epoch + epochs + 1):
    model.train()
    running_total = running_recon = running_kld = running_perceptual =  0.0
    progress_bar = tqdm(enumerate(train_loader, start=1), total=len(train_loader), desc="Training")

    for batch_idx, (x, _) in progress_bar:
        x = x.to(device, non_blocking=True)
        optimizer.zero_grad(set_to_none=True)
        logits, mean, logvar = model(x)
        loss, l1_loss, perc_loss, kl_loss = vgg_loss(logits, x, mean, logvar, func="mse")
        # loss, l1_loss, perc_loss, kl_loss = criterion(x_recon, x, mu, logvar)
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        progress_bar.set_postfix(
            loss=f"{loss.item():.3f}",
            l1=f"{l1_loss.item():.3f}",
            kld=f"{kl_loss.item():.3f}",
            percep=f"{perc_loss.item():.3f}",

        )
        running_total += loss.item()
        running_recon += l1_loss.item()
        running_kld += kl_loss.item()
        running_perceptual += perc_loss.item()
    current_epoch += 1
    print(
        f"Epoch {epoch:02d} | total: {running_total / n_train:.4f} | "
        f"recon: {running_recon / n_train:.4f} | kld: {running_kld / n_train:.4f} | "
        f"perceptual: {running_perceptual / n_train:.4f}"
    )

    save_reconstructions(model, x, out_dir, current_epoch, device)

        # print(
        #      f"Epoch {epoch:02d} | loss: {loss.item}
        # print(
        #     f"Epoch {epoch:02d} | total: {running_total / n_train:.4f} | "
        #     f"recon: {running_recon / n_train:.4f} | kld: {running_kld / n_train:.4f} | "
        #     f"beta: {beta:.4f}"
        # )
        # if(epoch % 10) == 0:
        #     training_loss_tracker.append([running_total / n_train, running_recon / n_train, running_kld / n_train, beta])

    if epoch % 100 == 0:
        save_checkpoint(model, optimizer, epoch, checkpoint_dir)

Training: 100%|██████████| 159/159 [00:18<00:00,  8.57it/s, kld=0.002, l1=0.812, loss=0.850, percep=0.036]


Epoch 01 | total: 0.0139 | recon: 0.0133 | kld: 0.0000 | perceptual: 0.0006


Training: 100%|██████████| 159/159 [00:17<00:00,  9.33it/s, kld=0.035, l1=0.704, loss=0.773, percep=0.034]


Epoch 02 | total: 0.0133 | recon: 0.0126 | kld: 0.0001 | perceptual: 0.0006


In [ ]:
epochs = 50
for epoch in range(current_epoch + 1, current_epoch + epochs + 1):
    model.train()
    running_total = running_recon = running_kld = running_perceptual =  0.0
    progress_bar = tqdm(enumerate(train_loader, start=1), total=len(train_loader), desc="Training")

    for batch_idx, (x, _) in progress_bar:
        x = x.to(device, non_blocking=True)
        optimizer.zero_grad(set_to_none=True)
        logits, mean, logvar = model(x)
        loss, l1_loss, perc_loss, kl_loss = vgg_loss(logits, x, mean, logvar)
        # loss, l1_loss, perc_loss, kl_loss = criterion(x_recon, x, mu, logvar)
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        progress_bar.set_postfix(
            loss=f"{loss.item():.3f}",
            l1=f"{l1_loss.item():.3f}",
            kld=f"{kl_loss.item():.3f}",
            percep=f"{perc_loss.item():.3f}",

        )
        running_total += loss.item()
        running_recon += l1_loss.item()
        running_kld += kl_loss.item()
        running_perceptual += perc_loss.item()
    current_epoch += 1
    print(
        f"Epoch {epoch:02d} | total: {running_total / n_train:.4f} | "
        f"recon: {running_recon / n_train:.4f} | kld: {running_kld / n_train:.4f} | "
        f"perceptual: {running_perceptual / n_train:.4f}"
    )

    save_reconstructions(model, x, out_dir, current_epoch, device)
    n_train = len(train_loader.dataset)

        # print(
        #      f"Epoch {epoch:02d} | loss: {loss.item}
        # print(
        #     f"Epoch {epoch:02d} | total: {running_total / n_train:.4f} | "
        #     f"recon: {running_recon / n_train:.4f} | kld: {running_kld / n_train:.4f} | "
        #     f"beta: {beta:.4f}"
        # )
        # if(epoch % 10) == 0:
        #     training_loss_tracker.append([running_total / n_train, running_recon / n_train, running_kld / n_train, beta])

    if epoch % 100 == 0:
        save_checkpoint(model, optimizer, epoch, checkpoint_dir)